In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error

import xgboost as xgb
from scripts.benchmark import DATA_DIR, DATASETS, config_data,neg_log_likelihood

In [ ]:
X_train, X_test, y_train, y_test = config_data("airfoil", split_seed=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


INFO:2026-01-15 19:54:46,792:jax._src.xla_bridge:810: Unable to initialize backend 'tpu': UNIMPLEMENTED: LoadPjrtPlugin is not implemented on windows yet.
2026-01-15 19:54:46,792 - jax._src.xla_bridge - INFO - Unable to initialize backend 'tpu': UNIMPLEMENTED: LoadPjrtPlugin is not implemented on windows yet.


(1202, 5)
(301, 5)
(1202,)
(301,)


In [ ]:
reg = xgb.XGBRegressor(
    tree_method="hist",
    n_estimators=2000,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=0,
)

reg.fit(
    X_train, y_train,
    verbose=False,
)

pred = reg.predict(X_test)
rmse = root_mean_squared_error(y_test, pred)
print("rmse", root_mean_squared_error(y_test, pred))
print(f"nll { neg_log_likelihood(y_test, pred, root_mean_squared_error(y_test, pred)).mean()}")

rmse 0.19470268487930298
nll -0.21734303442492023


In [ ]:
from sklearn.ensemble import RandomForestRegressor
mlp= RandomForestRegressor(n_estimators=300)
mlp.fit(X_train,y_train)
mlp_pred= mlp.predict(X_test)
print("rmse", root_mean_squared_error(y_test, mlp_pred))
print(f"nll { neg_log_likelihood(y_test, pred, root_mean_squared_error(y_test, mlp_pred)).mean()}")

rmse 0.26445523823769435
nll -0.14011962039061873


In [ ]:
from sklearn.linear_model import LinearRegression
mlp = LinearRegression(fit_intercept=False)
mlp.fit(X_train,y_train)
mlp_pred = mlp.predict(X_test)
print("rmse", root_mean_squared_error(y_test, mlp_pred))
print(f"nll { neg_log_likelihood(y_test, pred, root_mean_squared_error(y_test, mlp_pred)).mean()}")

rmse 0.6867814660072327
nll 0.5833855801014137


In [ ]:
from catboost import CatBoostRegressor
cat=CatBoostRegressor(depth=6, iterations=1000, learning_rate=0.05)
cat.fit(X_train,y_train)
cat_pred = cat.predict(X_test)
print("rmse", root_mean_squared_error(y_test, cat_pred))
print(f"nll { neg_log_likelihood(y_test, pred, root_mean_squared_error(y_test, cat_pred)).mean()}")

0:	learn: 0.9767437	total: 148ms	remaining: 2m 27s
1:	learn: 0.9549973	total: 149ms	remaining: 1m 14s
2:	learn: 0.9310037	total: 149ms	remaining: 49.7s
3:	learn: 0.9128200	total: 150ms	remaining: 37.4s
4:	learn: 0.8899329	total: 151ms	remaining: 30s
5:	learn: 0.8720766	total: 151ms	remaining: 25.1s
6:	learn: 0.8562042	total: 152ms	remaining: 21.6s
7:	learn: 0.8379556	total: 153ms	remaining: 18.9s
8:	learn: 0.8199306	total: 153ms	remaining: 16.9s
9:	learn: 0.8045970	total: 154ms	remaining: 15.2s
10:	learn: 0.7870368	total: 154ms	remaining: 13.9s
11:	learn: 0.7731777	total: 155ms	remaining: 12.8s
12:	learn: 0.7562408	total: 156ms	remaining: 11.8s
13:	learn: 0.7427784	total: 156ms	remaining: 11s
14:	learn: 0.7301996	total: 157ms	remaining: 10.3s
15:	learn: 0.7173943	total: 157ms	remaining: 9.68s
16:	learn: 0.7036907	total: 158ms	remaining: 9.14s
17:	learn: 0.6896145	total: 159ms	remaining: 8.67s
18:	learn: 0.6761310	total: 160ms	remaining: 8.25s
19:	learn: 0.6655755	total: 161ms	remaining

In [ ]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor(hidden_layer_sizes=[4, 16, 32, 63, 128, 256, 512, 256, 64, 8], solver="lbfgs", max_iter=5000)
mlp.fit(X_train, y_train)
mlp_pred = mlp.predict(X_test)
print("rmse", root_mean_squared_error(y_test, mlp_pred))
print(
    f"nll { neg_log_likelihood(y_test, pred, root_mean_squared_error(y_test, mlp_pred)).mean()}"
)

In [ ]:
from tabpfn import TabPFNRegressor
from tabpfn.constants import ModelVersion

tabpfn = TabPFNRegressor(random_state=2).create_default_for_version(
    ModelVersion.V2, ignore_pretraining_limits=True
)
tabpfn.fit(X_train, y_train)

tabpfn_pred = tabpfn.predict(X_test)

rmse = root_mean_squared_error(y_test, tabpfn_pred)
nll = neg_log_likelihood(y_test, tabpfn_pred, rmse).mean()

print("rmse", rmse)
print("nll", nll)

rmse 0.14475084841251373
nll -0.5138027585740881


In [ ]:
from bde import BdeRegressor
from bde.loss import GaussianNLL
from bde.sampler.prior import PriorDist
import pandas as pd 
import jax.numpy as jnp
from sklearn.preprocessing import StandardScaler

def regression_concrete_example():
    print("-" * 20)
    print("Regression example for concrete data")
    print("-" * 20)

    data = pd.read_csv("bde/data/concrete.data", sep=" ", header=None)

    X = data.iloc[:, :-1].values  # (1030, 8)
    y = data.iloc[:, -1].values  # (1030,)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    x_scaler = StandardScaler()
    y_scaler = StandardScaler()

    X_train = x_scaler.fit_transform(X_train)
    X_test = x_scaler.transform(X_test)

    y_train = y_scaler.fit_transform(y_train.reshape(-1, 1)).ravel()
    y_test = y_scaler.transform(y_test.reshape(-1, 1)).ravel()

    regressor = BdeRegressor(
        hidden_layers=[16, 16],
        n_members=8,
        seed=0,
        loss=GaussianNLL(),
        epochs=1000,
        validation_split=0.15,
        lr=1e-3,
        weight_decay=1e-4,
        warmup_steps=5000,
        n_samples=1000,
        n_thinning=0,
        patience=10,
        prior_family=PriorDist.NORMAL,
        prior_kwargs={"loc": 0, "scale": 1.5},
    )

    print(f"The params are {regressor.get_params()}")
    regressor.fit(x=jnp.array(X_train), y=jnp.array(y_train))

    means, sigmas = regressor.predict(jnp.array(X_test), mean_and_std=True)
    rmse = root_mean_squared_error(y_true=y_test, y_pred=means)
    print("RMSE:", rmse)
    mean, intervals = regressor.predict(X_test, credible_intervals=[0.1, 0.9])
    raw = regressor.predict(X_test, raw=True)
    print(
        f"The shape of the raw predictions are {raw.shape}"
    )  # (ensemble members, n_samples/n_thinning, n_data, (mu,sigma))

    # use the raw predictions to compute log pointwise predictive density (lppd)
    n_data = y_test.shape[0]
    log_likelihoods = norm.logpdf(
        y_test.reshape(1, 1, n_data),
        loc=raw[:, :, :, 0],
        scale=jax.nn.softplus(raw[..., 1]) + 1e-6,  # map raw scale via softplus
    )  # (E,T,N)
    b = 1 / jnp.prod(jnp.array(log_likelihoods.shape[:-1]))  # 1/ET
    axis = tuple(range(len(log_likelihoods.shape) - 1))
    log_likelihoods = jax.scipy.special.logsumexp(log_likelihoods, b=b, axis=axis)
    lppd = jnp.mean(log_likelihoods)
    print(f"The log pointwise predictive density (lppd) is {lppd}")

    print("Quantiles shape:", intervals.shape)  # (len(q), N)
    # calculate the coverage of the 80% credible interval
    lower = intervals[0]
    upper = intervals[1]
    coverage = jnp.mean((y_test.ravel() >= lower) & (y_test.ravel() <= upper))
    print(f"Coverage of the 80% credible interval: {coverage * 100:.2f}%")

    score = regressor.score(X_test, y_test)
    print(f"The sklearn score is {score}")

regression_concrete_example()

--------------------
Regression example for concrete data
--------------------
The params are {'activation': 'relu', 'desired_energy_var_end': 0.1, 'desired_energy_var_start': 0.5, 'epochs': 1000, 'hidden_layers': [16, 16], 'loss': <bde.loss.loss.GaussianNLL object at 0x000001812D453090>, 'lr': 0.001, 'n_members': 8, 'n_samples': 10000, 'n_thinning': 0, 'patience': 10, 'prior_family': <PriorDist.NORMAL: 'Normal'>, 'prior_kwargs': {'loc': 0, 'scale': 1.5}, 'seed': 0, 'step_size_init': None, 'validation_split': 0.15, 'warmup_steps': 50000, 'weight_decay': 0.0001}


2026-01-15 20:02:46,367 - bde.bde_builder - INFO - Kernel devices: 10
2026-01-15 20:02:48,248 - bde.bde_builder - INFO - Epoch 0: 10 ensemble members still training
2026-01-15 20:02:50,550 - bde.bde_builder - INFO - Epoch 100: 10 ensemble members still training
2026-01-15 20:02:51,834 - bde.bde_builder - INFO - Epoch 200: 10 ensemble members still training
2026-01-15 20:02:53,101 - bde.bde_builder - INFO - Epoch 300: 10 ensemble members still training
2026-01-15 20:02:54,409 - bde.bde_builder - INFO - Epoch 400: 5 ensemble members still training
2026-01-15 20:02:55,630 - bde.bde_builder - INFO - Epoch 500: 4 ensemble members still training
2026-01-15 20:02:56,952 - bde.bde_builder - INFO - Epoch 600: 2 ensemble members still training
2026-01-15 20:02:58,269 - bde.bde_builder - INFO - Epoch 700: 1 ensemble members still training
2026-01-15 20:02:59,535 - bde.bde_builder - INFO - Epoch 800: 1 ensemble members still training
2026-01-15 20:02:59,864 - bde.bde_builder - INFO - All members s